In [99]:
import pandas as pd
import re
from datetime import datetime

In [100]:
file_path = 'ICM Excel Sheet.xlsx'

In [101]:
df = pd.read_excel(file_path)
#df.head()

In [102]:
# Select specific columns
df1 = df[['UID', 'BudgetItem', 'Department', 'Amount', 'Vendor/Transfer to department', 'AmountRemarks']]
#df1.head()

In [103]:
# Rename specific columns
df2 = df1.rename(columns={'UID': 'Entry No', 'BudgetItem': 'Dr Ledger Name', 'Department':'Dr Cost Center', 'Amount':'Dr Amt', 'Vendor/Transfer to department':'Cr Ledger Name' })
#df2.head()

In [104]:
# Modified function to extract Bill No., Invoice No., or Quotation Ref No.
def extract_reference(text):
    bill_match = re.search(r'Bill No\.\s*(\S+)', text)
    invoice_match = re.search(r'Invoice No\.\s*(\S+)', text)
    quotation_match = re.search(r'Quotation Ref No\.\s*(\S+)', text)
    
    if bill_match:
        return bill_match.group(1)
    elif invoice_match:
        return invoice_match.group(1)
    elif quotation_match:
        return quotation_match.group(1)
    
    return None

# Apply the function to your data frame
df2['Bill Ref No.'] = df2['AmountRemarks'].apply(extract_reference)

# Display the first 50 rows
#df2.head()

In [105]:
# Create the new 'Vch Narration' column by merging 'Entry No.' and 'AmountRemarks'
df2['Vch Narration'] = 'UID No. ' + df2['Entry No'].astype(str) + ' ' + df2['AmountRemarks']
# Display the updated dataframe
#df2.head()

In [106]:
df3=df2[['Entry No', 'Dr Ledger Name',	'Dr Cost Center', 'Dr Amt', 'Cr Ledger Name', 'Bill Ref No.', 'Vch Narration' ]]
#df3.head()

In [107]:
# Adding blank columns at specific positions
df3.insert(0, 'Date', '')  # Add 'Date' at the first position
df3.insert(2, 'Vch Name', '')  # Add 'Vch No' at the third position
df3.insert(7, 'Cr Cost Center', '')  # Add 'cost center' at the eighth position
df3.insert(8, 'Cr Amt', '')  # Add 'cost center' at the ninth position
# Display the updated dataframe
df3.head(5)

,Date,Entry No,Vch Name,Dr Ledger Name,Dr Cost Center,Dr Amt,Cr Ledger Name,Cr Cost Center,Cr Amt,Bill Ref No.,Vch Narration
0,,4250005193,,ADMN-Administrative Expenses,RDD-REP-AZTECH,3000,Govardhan Eco Village Trust (SBM),,,GV/GF/I/2500041,UID No. 4250005193 Payment for Office Space Re...
1,,4250005193,,ADMN-Administrative Expenses,RDD-Wadi-NEO,1500,Govardhan Eco Village Trust (SBM),,,GV/GF/I/2500041,UID No. 4250005193 Payment for Office Space Re...
2,,4250005193,,ADMN-Administrative Expenses,RDD-SFL-Praj HiPurity,3000,Govardhan Eco Village Trust (SBM),,,GV/GF/I/2500041,UID No. 4250005193 Payment for Office Space Re...
3,,4250005193,,ADMN-Administrative Expenses,RDD-WADI-A-NOCIL-Yr-2021-22,2000,Govardhan Eco Village Trust (SBM),,,GV/GF/I/2500041,UID No. 4250005193 Payment for Office Space Re...
4,,4250005193,,ADMN-Administrative Expenses,RDD-MARKETING-LINK-NOCIL,3300,Govardhan Eco Village Trust (SBM),,,GV/GF/I/2500041,UID No. 4250005193 Payment for Office Space Re...


In [108]:
df4 = df3[['Date','Entry No','Vch Name','Dr Ledger Name','Dr Amt','Dr Cost Center','Cr Ledger Name','Cr Amt','Cr Cost Center','Vch Narration','Bill Ref No.']]
df4.head(5)

,Date,Entry No,Vch Name,Dr Ledger Name,Dr Amt,Dr Cost Center,Cr Ledger Name,Cr Amt,Cr Cost Center,Vch Narration,Bill Ref No.
0,,4250005193,,ADMN-Administrative Expenses,3000,RDD-REP-AZTECH,Govardhan Eco Village Trust (SBM),,,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041
1,,4250005193,,ADMN-Administrative Expenses,1500,RDD-Wadi-NEO,Govardhan Eco Village Trust (SBM),,,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041
2,,4250005193,,ADMN-Administrative Expenses,3000,RDD-SFL-Praj HiPurity,Govardhan Eco Village Trust (SBM),,,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041
3,,4250005193,,ADMN-Administrative Expenses,2000,RDD-WADI-A-NOCIL-Yr-2021-22,Govardhan Eco Village Trust (SBM),,,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041
4,,4250005193,,ADMN-Administrative Expenses,3300,RDD-MARKETING-LINK-NOCIL,Govardhan Eco Village Trust (SBM),,,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041


In [109]:
# Ensure 'Dr Amt' is a string and remove commas
df3['Dr Amt'] = df3['Dr Amt'].astype(str).str.replace(',', '', regex=True)
# Convert 'Dr Amt' to numeric
df3['Dr Amt'] = pd.to_numeric(df3['Dr Amt'])
# Create a copy of df3 to df4
df4 = df3.copy()
# Calculate cumulative sum for each 'Entry No'
df4['Cr Amt'] = df3.groupby('Entry No')['Dr Amt'].transform('sum')
# Format 'Dr Amt' and 'Cr Amt' with commas and two decimal places
df4['Dr Amt'] = df4['Dr Amt'].apply(lambda x: f"{x:,.2f}")
df4['Cr Amt'] = df4['Cr Amt'].apply(lambda x: f"{x:,.2f}")
# Display the final DataFrame
df4.head(5)

C:\Users\grd00\AppData\Local\Temp\ipykernel_28536\2643644666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Dr Amt'] = df3['Dr Amt'].astype(str).str.replace(',', '', regex=True)
C:\Users\grd00\AppData\Local\Temp\ipykernel_28536\2643644666.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Dr Amt'] = pd.to_numeric(df3['Dr Amt'])


,Date,Entry No,Vch Name,Dr Ledger Name,Dr Cost Center,Dr Amt,Cr Ledger Name,Cr Cost Center,Cr Amt,Bill Ref No.,Vch Narration
0,,4250005193,,ADMN-Administrative Expenses,RDD-REP-AZTECH,"3,000.00",Govardhan Eco Village Trust (SBM),,"21,600.00",GV/GF/I/2500041,UID No. 4250005193 Payment for Office Space Re...
1,,4250005193,,ADMN-Administrative Expenses,RDD-Wadi-NEO,"1,500.00",Govardhan Eco Village Trust (SBM),,"21,600.00",GV/GF/I/2500041,UID No. 4250005193 Payment for Office Space Re...
2,,4250005193,,ADMN-Administrative Expenses,RDD-SFL-Praj HiPurity,"3,000.00",Govardhan Eco Village Trust (SBM),,"21,600.00",GV/GF/I/2500041,UID No. 4250005193 Payment for Office Space Re...
3,,4250005193,,ADMN-Administrative Expenses,RDD-WADI-A-NOCIL-Yr-2021-22,"2,000.00",Govardhan Eco Village Trust (SBM),,"21,600.00",GV/GF/I/2500041,UID No. 4250005193 Payment for Office Space Re...
4,,4250005193,,ADMN-Administrative Expenses,RDD-MARKETING-LINK-NOCIL,"3,300.00",Govardhan Eco Village Trust (SBM),,"21,600.00",GV/GF/I/2500041,UID No. 4250005193 Payment for Office Space Re...


In [110]:
df4 = df4[['Date','Entry No','Vch Name','Dr Ledger Name','Dr Amt','Dr Cost Center','Cr Ledger Name', 'Cr Amt', 'Cr Cost Center', 'Vch Narration','Bill Ref No.']]
from datetime import datetime

# Fill 'Date' column with today's date in DD/MM/YYYY format
df4['Date'] = datetime.today().strftime('%d/%m/%Y')

# Fill 'Vch Name' column with "Journal"
df4['Vch Name'] = 'Journal'

df4.tail()

,Date,Entry No,Vch Name,Dr Ledger Name,Dr Amt,Dr Cost Center,Cr Ledger Name,Cr Amt,Cr Cost Center,Vch Narration,Bill Ref No.
52,04/06/2025,4250000877,Journal,ADMN-Administrative Expenses,130.00,RDD-REP-AZTECH,MSEDCL-003464,130.00,,UID No. 4250000877 Electricity Bill Charges fo...,000002815538064
53,04/06/2025,4250000876,Journal,ADMN-Administrative Expenses,130.00,RDD-REP-AZTECH,MSEDCL-003464,130.00,,UID No. 4250000876 Electricity Bill Charges fo...,000002815538062
54,04/06/2025,4250000875,Journal,ADMN-Administrative Expenses,200.00,RDD-REP-AZTECH,MSEDCL-003464,200.00,,UID No. 4250000875 Electricity Bill Charges fo...,000002815538298
55,04/06/2025,4250000872,Journal,PRM-Food Expenses,340.00,RDD-REP-Vidyavan (Gauranga Prabhu),Mahalaxmi Cloth Store Pro Jain,340.00,,UID No. 4250000872 Purchases of Carry Bags for...,074
56,04/06/2025,4250000871,Journal,PRM-Food Expenses,340.00,RDD-REP-Vidyavan (Gauranga Prabhu),Mahalaxmi Cloth Store Pro Jain,340.00,,UID No. 4250000871 Purchases of Carry Bags for...,086


In [111]:
df5 = df4.drop_duplicates()
df5 = df5[~df5.apply(lambda row: row.astype(str).str.contains('Shri Chaitanya Health and Care Trust \(Branch Transfer\)').any(), axis=1)]

# Replace blank or NaN values in 'Bill Ref No.' with values from 'Entry No'
df5['Bill Ref No.'] = df5['Bill Ref No.'].replace('', pd.NA)  # convert empty strings to NA
df5['Bill Ref No.'] = df5['Bill Ref No.'].fillna(df5['Entry No'])
df5


<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\grd00\AppData\Local\Temp\ipykernel_28536\3604717973.py:2: SyntaxWarning: invalid escape sequence '\('
  df5 = df5[~df5.apply(lambda row: row.astype(str).str.contains('Shri Chaitanya Health and Care Trust \(Branch Transfer\)').any(), axis=1)]


,Date,Entry No,Vch Name,Dr Ledger Name,Dr Amt,Dr Cost Center,Cr Ledger Name,Cr Amt,Cr Cost Center,Vch Narration,Bill Ref No.
0,04/06/2025,4250005193,Journal,ADMN-Administrative Expenses,"3,000.00",RDD-REP-AZTECH,Govardhan Eco Village Trust (SBM),"21,600.00",,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041
1,04/06/2025,4250005193,Journal,ADMN-Administrative Expenses,"1,500.00",RDD-Wadi-NEO,Govardhan Eco Village Trust (SBM),"21,600.00",,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041
2,04/06/2025,4250005193,Journal,ADMN-Administrative Expenses,"3,000.00",RDD-SFL-Praj HiPurity,Govardhan Eco Village Trust (SBM),"21,600.00",,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041
3,04/06/2025,4250005193,Journal,ADMN-Administrative Expenses,"2,000.00",RDD-WADI-A-NOCIL-Yr-2021-22,Govardhan Eco Village Trust (SBM),"21,600.00",,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041
4,04/06/2025,4250005193,Journal,ADMN-Administrative Expenses,"3,300.00",RDD-MARKETING-LINK-NOCIL,Govardhan Eco Village Trust (SBM),"21,600.00",,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041
5,04/06/2025,4250005193,Journal,ADMN-Administrative Expenses,"6,800.00",RDD-SFL-NOCIL-Yr-2-2021-22,Govardhan Eco Village Trust (SBM),"21,600.00",,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041
6,04/06/2025,4250005193,Journal,ADMN-Administrative Expenses,"2,000.00",RDD-WADI-MO,Govardhan Eco Village Trust (SBM),"21,600.00",,UID No. 4250005193 Payment for Office Space Re...,GV/GF/I/2500041
7,04/06/2025,4250005160,Journal,ADMN-Administrative Expenses,"4,500.00",RDD-SFL-NOCIL-Yr-2-2021-22,SHREE GANESH MACHINERY STORES,"8,500.00",,UID No. 4250005160 Purchase of Lubi Water Pres...,SGMS/0892/25-26
8,04/06/2025,4250005160,Journal,ADMN-Administrative Expenses,"4,000.00",RDD-WADI-A-NOCIL-Yr-2021-22,SHREE GANESH MACHINERY STORES,"8,500.00",,UID No. 4250005160 Purchase of Lubi Water Pres...,SGMS/0892/25-26
47,04/06/2025,4250005087,Journal,PERL-Accountant Salary,"5,250.00",RDD-WADI-MO,Manishkumar Mungatilal Verma,"47,505.00",,UID No. 4250005087 Salary of Accountant for th...,4250005087


In [112]:

# Get current date and timestamp
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# Create output file name with date and timestamp
output_filename = f'ICM_to_TDL_Sheet_{current_time}.xlsx'
# Convert DataFrame to Excel
df5.to_excel(output_filename, index=False)